In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 49.0 MB/s 


In [1]:
!pwd

/content


In [2]:
!cp '/content/drive/MyDrive/requirements.txt' .

In [3]:
!ls

drive  requirements.txt  sample_data


In [4]:
!pip install -r requirements.txt

In [5]:
!pip freeze | grep konlpy

konlpy==0.6.0


In [7]:
import tensorflow as tf
import numpy as np
import os
import json

from tensorflow.keras.layers import Layer, Embedding, GRU

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

import sys
sys.path.append("/content/drive/MyDrive/")

from preprocess import *

In [8]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [9]:
DATA_IN_PATH = '/content/drive/MyDrive/data_in/'
DATA_OUT_PATH = '/content/drive/MyDrive/data_out/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

In [10]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

In [11]:
index_inputs = np.load(open(DATA_IN_PATH + TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'rb'))
index_targets = np.load(open(DATA_IN_PATH + TRAIN_TARGETS , 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [12]:
print(len(index_inputs),  len(index_outputs), len(index_targets))

20 20 20


In [13]:
MODEL_NAME = 'seq2seq_kor'
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCH = 30
UNITS = 1024
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.1 

char2idx = prepro_configs['char2idx']
idx2char = prepro_configs['idx2char']
std_index = prepro_configs['std_symbol']
end_index = prepro_configs['end_symbol']
vocab_size = prepro_configs['vocab_size']

In [14]:
class Encoder(Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size

        self.embedding = Embedding(self.vocab_size, self.embedding_dim)
        self.gru = GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
      x = self.embedding(x)
      output, state = self.gru(x, initial_state=hidden)
      return output, state

    def initialize_hidden_state(self, inp):
      
      return tf.zeros((tf.shape(inp)[0], self.enc_units))

In [15]:
encoder = Encoder(vocab_size, EMBEDDING_DIM, UNITS, BATCH_SIZE)

In [16]:
index_inputs.shape

(20, 25)

In [17]:
enc_hidden = encoder.initialize_hidden_state(index_inputs)
enc_hidden.shape

TensorShape([20, 1024])

Bahdanau Attention

In [18]:
class BahdanauAttention(Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()

    self.W1 = Dense(units) #입력과 w
    self.W2 = Dense(units)
    self.V = Dense(1) #output

  def call(self, query, values):
    # expand_dim: 
    hidden_with_time_axis = tf.expand_dim(query, 1)

    #하이퍼볼릭 함수 지나도록 함
    score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

    #softmax
    attention_weights = tf.nn.softmax(score, axis=1)

    #softmax한 걸 기존거랑 곱해줘야 함: context_vector
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


디코더(seq2seq 교안 p.15)

In [19]:
class Decoder(Layer):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()

    self.batch_sz = batch_sz
    self.decoder_units = dec_units
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim

    self.embedding = Embedding(self.vocab_size, self.embedding_dim)
    #GRU: 입력으로 들어온 애가 타임스텝별로 그 아웃풋 사용
    self.gru = GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

    # hidden state 나온 걸로 단어 맞춰야 함
    self.fc = Dense(self.vocab_size) #출력을 vocab_size -> softmax -> crossentropy -> 가장 높은 라벨 중 하나를 고름

    self.attention = BahdanauAttention(self.dec_units)

  #x: decoder의 input, hidden: 어텐션 스코어, enc_output: 마지막 단계
  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)

    x = self.embedding(x)

    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    #입력이 들어가기 전에 어텐션 거는 것
    #교안예시는 입력이 들어간 다음, 나중에 어텐션을 걸었음
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2])) #shape 맞추기

    x = self.fc(output)

    return x, state, attention_weights